In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle 
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

2.13.0


In [2]:
## loading the model
model = load_model("retention_risk_scoring_model.keras")

## loading the encoder and scaler
with open("label_encoder_gender.pkl", "rb") as f:
    label_encoder_gender = pickle.load(f)

with open("onehot_encoder_geography.pkl", "rb") as f:
    onehot_encoder_geography = pickle.load(f)

with open("scaler.pkl", "rb") as f:
    scaler = pickle.load(f)

In [3]:
## input data
input_data = {
    "CreditScore": 600,
    "Geography": "France",
    "Gender": "Male",
    "Age": 35,
    "Tenure": 4,
    "Balance": 77000,
    "NumOfProducts": 2,
    "HasCrCard": 1,
    "IsActiveMember": 1,
    "EstimatedSalary": 50000
}

In [4]:
input_data = pd.DataFrame([input_data])

In [5]:
## converting categorical features into numerical
geo_encoded = onehot_encoder_geography.transform(input_data[["Geography"]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geography.get_feature_names_out(["Geography"]))

In [6]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [7]:
input_data["Gender"] = label_encoder_gender.transform(input_data["Gender"])

In [8]:
## concatenation of ohe features with input data
input_data = pd.concat([input_data.drop("Geography", axis = 1), geo_encoded_df], axis=1)

In [9]:
input_data.dtypes

CreditScore            int64
Gender                 int32
Age                    int64
Tenure                 int64
Balance                int64
NumOfProducts          int64
HasCrCard              int64
IsActiveMember         int64
EstimatedSalary        int64
Geography_France     float64
Geography_Germany    float64
Geography_Spain      float64
dtype: object

In [10]:
## scaling the features
input_data_scaled = scaler.transform(input_data)
input_data_scaled

array([[-0.54007722,  0.91186722, -0.36939437, -0.34496951,  0.01184899,
         0.81311138,  0.64946204,  0.96978946, -0.88017893,  1.00053348,
        -0.57776083, -0.57735027]])

In [11]:
##making predictions
predictions = model.predict(input_data_scaled)
predictions

1/1 [==============================] - 1s 592ms/step


array([[0.03111878]], dtype=float32)

In [13]:
predictions_probability = predictions[0][0]

In [14]:
if predictions_probability > 0.5:
    print("The customer is likely to leave the bank.")
else:
    print("The customer is likely to stay with the bank.")

The customer is likely to stay with the bank.
